# モジュールをインポート

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# モデルを定義

In [3]:
from tensorflow.keras.models import load_model

In [4]:
results_dir = "./results/keras_2019_12_23_12_11"

In [5]:
model = load_model(results_dir + "/model/model.h5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# モデルの確認

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10, 10, 11, 1)]   0         
_________________________________________________________________
conv3d (Conv3D)              (None, 10, 10, 11, 128)   1280      
_________________________________________________________________
reshape (Reshape)            (None, 10, 14080)         0         
_________________________________________________________________
dense (Dense)                (None, 10, 1024)          14418944  
_________________________________________________________________
dropout (Dropout)            (None, 10, 1024)          0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 10, 1024)          0         
_________________________________________________________________
gru (GRU)                    (None, 1024)              629452

In [7]:
# opt = Adam(lr=learning_rate)
# model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

## 学習済みモデルの重みを読み込み

In [8]:
# checkpoint_path = "./results/keras_2019_11_21/keras_model/cp.ckpt"
# model.load_weights(checkpoint_path)

# Load data stream

In [9]:
# from pylsl import StreamInlet, resolve_byprop, local_clock
# from muselsl.constants import LSL_SCAN_TIMEOUT, LSL_EEG_CHUNK
# from datetime import datetime
# LSL_EEG_CHUNK = 10
# def record_stream():
#     """
#     :return: a generator to fetch EEG data from existed stream.
#     """
#     streams = resolve_byprop("type", "EEG", timeout=LSL_SCAN_TIMEOUT)
#     if len(streams) == 0:
#         raise IOError("Can't find EEG stream.")

#     inlet = StreamInlet(streams[0], max_buflen=LSL_EEG_CHUNK)

#     while True:
#         chunk, timestamp = inlet.pull_sample(timeout=1.0)
#         if timestamp:
#             timestamp = datetime.fromtimestamp(timestamp).strftime("%A, %B %d, %Y %I:%M:%S")
#             print(timestamp, end="\r", flush=True)
# record_stream()

In [10]:
from pylsl import StreamInlet, resolve_byprop
from muselsl.constants import LSL_SCAN_TIMEOUT, LSL_EEG_CHUNK

LSL_EEG_CHUNK = 10
def record_stream():
    """
    :return: a generator to fetch EEG data from existed stream.
    """
    streams = resolve_byprop("type", "EEG", timeout=LSL_SCAN_TIMEOUT)
    if len(streams) == 0:
        raise IOError("Can't find EEG stream.")

    inlet = StreamInlet(streams[0], max_buflen=LSL_EEG_CHUNK)

    while True:
        yield inlet.pull_chunk(timeout=1.0, max_samples=LSL_EEG_CHUNK)


In [11]:
samples, timestamps = next(record_stream())
print(samples)

[[353.515625, 28.80859375, 30.2734375, 855.95703125, 61.03515625], [94.7265625, 34.66796875, 41.50390625, 347.65625, 81.54296875], [-274.4140625, 39.55078125, 40.52734375, -700.68359375, -20.5078125], [-119.140625, 36.62109375, 30.76171875, -474.609375, -32.2265625], [312.01171875, 30.2734375, 28.80859375, 655.76171875, 45.41015625], [229.4921875, 35.64453125, 37.109375, 615.234375, 55.17578125], [-208.49609375, 43.45703125, 43.45703125, -521.97265625, 17.578125], [-213.8671875, 38.57421875, 35.64453125, -667.96875, 13.18359375], [196.2890625, 27.83203125, 25.87890625, 364.74609375, 46.38671875], [318.359375, 29.78515625, 34.66796875, 830.078125, 93.26171875]]


In [12]:
print(timestamps)

[1577257225.0890708, 1577257225.092977, 1577257225.0968833, 1577257225.1007895, 1577257225.1046958, 1577257225.108602, 1577257225.1125083, 1577257225.1164145, 1577257225.1203208, 1577257225.124227]


In [13]:
# print("['TP9', 'AF7', 'AF8', 'TP10', 'Right AUX']")
# print(np.array(next(record_stream())[0])[:, 0:4])

# Preprocessing

In [14]:
def data_1Dto2D(data, Y=10, X=11):
    data_2D = np.zeros([Y, X])
    data_2D[1, 3] = data[1]
    data_2D[1, 7] = data[2]
    data_2D[5, 1] = data[0]
    data_2D[5, 9] = data[3]
# 	data_2D[0] = ( 	   	 0, 	   0,  	   	 0, 	   0,        0,        0,        0, 	   0,  	     0, 	   0, 	 	 0) 
# 	data_2D[1] = (	  	 0, 	   0,  	   	 0, data[1],        0,        0,        0, data[2], 	   	 0,   	   0, 	 	 0) 
# 	data_2D[2] = (	  	 0,        0,        0,        0,        0,        0,        0,        0,        0,        0, 	 	 0) 
# 	data_2D[3] = (	  	 0,        0,        0,        0,        0,        0,        0,        0,        0,        0, 		 0) 
# 	data_2D[4] = (       0,        0,        0,        0,        0,        0,        0,        0,        0,        0,        0) 
# 	data_2D[5] = (	  	 0, data[0],        0,        0,        0,        0,        0,        0,        0, data[3], 		 0) 
# 	data_2D[6] = (	  	 0,        0,        0,        0,        0,        0,        0,        0,        0,        0, 		 0) 
# 	data_2D[7] = (	  	 0, 	   0, 	 	 0,        0,        0,        0,        0,        0, 	   	 0, 	   0, 		 0) 
# 	data_2D[8] = (	  	 0, 	   0, 	 	 0, 	   0,        0,        0,        0, 	   0, 	   	 0, 	   0, 		 0) 
# 	data_2D[9] = (	  	 0, 	   0, 	 	 0, 	   0, 	     0,        0, 		 0, 	   0, 	   	 0, 	   0, 		 0) 
    return data_2D

In [15]:
true_labels = np.array(['eye_close', 'image_open&close_both_feet',
       'image_open&close_both_fists', 'image_open&close_left_fist',
       'image_open&close_right_fist'])

In [16]:
def feature_normalize(data):
	mean = data[data.nonzero()].mean()
	sigma = data[data.nonzero()].std()
	data_normalized = data
	data_normalized[data_normalized.nonzero()] = (data_normalized[data_normalized.nonzero()] - mean)/sigma
	return data_normalized

In [17]:
from IPython.display import clear_output
# print("23:51:39:746923  [[False False  True False False]]")
# print("   Timestamp##################Results#############")
print("Timestamp          Results")

for stream, timestamps in record_stream():
    buffer_3d = []
    buffer_2d = np.zeros((10, 11))
    num = 0
    for data in stream:
        data = np.array(data)
        data = feature_normalize(data)
        buffer_2d[1, 3] = data[1]
        buffer_2d[1, 7] = data[2]
        buffer_2d[5, 1] = data[0]
        buffer_2d[5, 9] = data[3]
        buffer_3d.append(buffer_2d)
    buffer_3d = np.array(buffer_3d)
    X_test = buffer_3d.reshape((1, 10, 10, 11, 1))
    y_pred = model.predict(X_test, batch_size=1, verbose=0)
    y_pred_bool = np.argmax(y_pred, axis=1)
#     print(y_pred.round(3), end="\r", flush=True)
#     y_pred = (y_pred > 0.8)
#     print(datetime.fromtimestamp(timestamps[-1]).strftime("%H:%M:%S:%f")
#           + "  " + str(y_pred), end="\r", flush=True)
    print(datetime.fromtimestamp(timestamps[-1]).strftime("%H:%M:%S:%f")
          + "  " + str(true_labels[int(y_pred_bool)]), end="\r", flush=True)
#     print(y_pred, end="\n")
#     print(flush=True)
#     y_pred_bool = np.argmax(y_pred, axis=1)
#     print(true_labels[int(y_pred_bool)], end="\r", flush=True)
#     print(true_labels[int(y_pred_bool)])

Timestamp          Results


KeyboardInterrupt: 